In [193]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar -xvf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark==3.5.1
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Colab") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

# Print the SparkSession variable
spark

spark-3.5.1-bin-hadoop3/
spark-3.5.1-bin-hadoop3/sbin/
spark-3.5.1-bin-hadoop3/sbin/spark-config.sh
spark-3.5.1-bin-hadoop3/sbin/stop-slave.sh
spark-3.5.1-bin-hadoop3/sbin/stop-mesos-dispatcher.sh
spark-3.5.1-bin-hadoop3/sbin/start-workers.sh
spark-3.5.1-bin-hadoop3/sbin/start-slaves.sh
spark-3.5.1-bin-hadoop3/sbin/start-all.sh
spark-3.5.1-bin-hadoop3/sbin/stop-all.sh
spark-3.5.1-bin-hadoop3/sbin/workers.sh
spark-3.5.1-bin-hadoop3/sbin/start-mesos-dispatcher.sh
spark-3.5.1-bin-hadoop3/sbin/spark-daemon.sh
spark-3.5.1-bin-hadoop3/sbin/decommission-worker.sh
spark-3.5.1-bin-hadoop3/sbin/slaves.sh
spark-3.5.1-bin-hadoop3/sbin/stop-mesos-shuffle-service.sh
spark-3.5.1-bin-hadoop3/sbin/stop-history-server.sh
spark-3.5.1-bin-hadoop3/sbin/stop-worker.sh
spark-3.5.1-bin-hadoop3/sbin/decommission-slave.sh
spark-3.5.1-bin-hadoop3/sbin/stop-thriftserver.sh
spark-3.5.1-bin-hadoop3/sbin/start-worker.sh
spark-3.5.1-bin-hadoop3/sbin/stop-slaves.sh
spark-3.5.1-bin-hadoop3/sbin/start-connect-server.sh


In [194]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [271]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [197]:
traindf = pd.read_csv('https://raw.githubusercontent.com/Harshit-Budakoti/enigma_hb/main/train.csv')

testdf = pd.read_csv('https://raw.githubusercontent.com/Harshit-Budakoti/enigma_hb/main/test.csv')

In [199]:
traindf.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARITAL_STATUS,AGE,INSTALLMENT_1,INSTALLMENT_2,INSTALLMENT_3,INSTALLMENT_AMT_1,INSTALLMENT_AMT_2,INSTALLMENT_AMT_3,INSTALLMENT_PAID_1,INSTALLMENT_PAID_2,INSTALLMENT_PAID_3,DEFAULT.PAYMENT.NEXT.MONTH
0,12194,220000.0,FEMALE,graduate school,MARRIED,54,1,-2,-1,0.0,0.0,788.0,0.0,788.0,0.0,0
1,7068,90000.0,FEMALE,graduate school,SINGLE,30,2,2,3,750.0,750.0,750.0,0.0,0.0,0.0,0
2,1446,80000.0,FEMALE,university,SINGLE,25,2,3,2,69602.0,67930.0,72205.0,0.0,6000.0,2700.0,0
3,318,260000.0,FEMALE,high school,MARRIED,53,1,-1,-1,0.0,165.0,526.0,165.0,526.0,452.0,0
4,13732,30000.0,FEMALE,university,SINGLE,23,0,0,0,25287.0,26501.0,30300.0,2011.0,5000.0,868.0,0


In [200]:
traindf = spark.createDataFrame(traindf)
testdf = spark.createDataFrame(testdf)

In [201]:
trdf = traindf.toPandas()

In [202]:
trdf.describe()

,ID,LIMIT_BAL,AGE,INSTALLMENT_1,INSTALLMENT_2,INSTALLMENT_3,INSTALLMENT_AMT_1,INSTALLMENT_AMT_2,INSTALLMENT_AMT_3,INSTALLMENT_PAID_1,INSTALLMENT_PAID_2,INSTALLMENT_PAID_3,DEFAULT.PAYMENT.NEXT.MONTH
count,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,21000.000000,2.100000e+04,21000.000000,2.100000e+04,21000.000000,21000.000000
mean,14989.511667,168351.714286,35.531524,-0.011381,-0.129619,-0.164714,51601.479000,49411.493524,4.726970e+04,5733.337667,5.980440e+03,5296.636048,0.223429
std,8671.132777,130145.303418,9.234924,1.131404,1.203697,1.201243,74247.117541,71506.968881,7.006471e+04,17411.603190,2.525646e+04,18060.540666,0.416553
min,1.000000,10000.000000,21.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-33350.000000,-6.150600e+04,0.000000,0.000000e+00,0.000000,0.000000
25%,7487.750000,50000.000000,28.000000,-1.000000,-1.000000,-1.000000,3490.000000,2898.500000,2.550750e+03,916.750000,7.827500e+02,390.000000,0.000000
50%,14980.500000,140000.000000,34.000000,0.000000,0.000000,0.000000,22578.000000,21190.500000,2.003800e+04,2100.000000,2.007000e+03,1836.500000,0.000000
75%,22492.250000,240000.000000,41.000000,0.000000,0.000000,0.000000,67435.750000,64366.500000,6.038625e+04,5005.000000,5.000000e+03,4600.000000,0.000000
max,30000.000000,1000000.000000,79.000000,8.000000,7.000000,8.000000,964511.000000,983931.000000,1.664089e+06,873552.000000,1.684259e+06,896040.000000,1.000000


In [203]:
traindf2 = traindf.withColumnRenamed('DEFAULT.PAYMENT.NEXT.MONTH', 'default')

In [204]:
traindf2.describe().show()

+-------+------------------+------------------+------+----------+-----------------+-----------------+--------------------+-------------------+-------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+
|summary|                ID|         LIMIT_BAL|   SEX| EDUCATION|   MARITAL_STATUS|              AGE|       INSTALLMENT_1|      INSTALLMENT_2|      INSTALLMENT_3|INSTALLMENT_AMT_1|INSTALLMENT_AMT_2|INSTALLMENT_AMT_3|INSTALLMENT_PAID_1|INSTALLMENT_PAID_2|INSTALLMENT_PAID_3|            default|
+-------+------------------+------------------+------+----------+-----------------+-----------------+--------------------+-------------------+-------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+
|  count|             21000|             21000| 21000|     21000|            21000|            21000|               21

In [205]:
trdf.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARITAL_STATUS', 'AGE',
       'INSTALLMENT_1', 'INSTALLMENT_2', 'INSTALLMENT_3', 'INSTALLMENT_AMT_1',
       'INSTALLMENT_AMT_2', 'INSTALLMENT_AMT_3', 'INSTALLMENT_PAID_1',
       'INSTALLMENT_PAID_2', 'INSTALLMENT_PAID_3',
       'DEFAULT.PAYMENT.NEXT.MONTH'],
      dtype='object')

In [206]:
def risk_rank_categorical_columns(df, i):
  from pyspark.sql.window import Window
  from pyspark.sql.functions import sum,avg
  window_spec = Window.partitionBy(i)
  result = df.withColumn('default_cases', sum('default').over(window_spec))
  res_table = result.select(i, 'default_cases').distinct().orderBy('default_cases')
  from pyspark.sql.functions import monotonically_increasing_id
  res_table = res_table.withColumn("DEFAULT_RISK_RANK", monotonically_increasing_id()+1)
  res_table.show()

In [207]:
req_columns = ['SEX','EDUCATION','MARITAL_STATUS']

In [208]:
traindf.show()

+-----+---------+------+---------------+--------------+---+-------------+-------------+-------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+--------------------------+
|   ID|LIMIT_BAL|   SEX|      EDUCATION|MARITAL_STATUS|AGE|INSTALLMENT_1|INSTALLMENT_2|INSTALLMENT_3|INSTALLMENT_AMT_1|INSTALLMENT_AMT_2|INSTALLMENT_AMT_3|INSTALLMENT_PAID_1|INSTALLMENT_PAID_2|INSTALLMENT_PAID_3|DEFAULT.PAYMENT.NEXT.MONTH|
+-----+---------+------+---------------+--------------+---+-------------+-------------+-------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+--------------------------+
|12194| 220000.0|FEMALE|graduate school|       MARRIED| 54|            1|           -2|           -1|              0.0|              0.0|            788.0|               0.0|             788.0|               0.0|                         0|
| 7068|  90000.0|FEMALE|graduate school|

In [209]:
traindf = traindf.withColumnRenamed('DEFAULT.PAYMENT.NEXT.MONTH','default')

In [210]:
traindf.dtypes

[('ID', 'bigint'),
 ('LIMIT_BAL', 'double'),
 ('SEX', 'string'),
 ('EDUCATION', 'string'),
 ('MARITAL_STATUS', 'string'),
 ('AGE', 'bigint'),
 ('INSTALLMENT_1', 'bigint'),
 ('INSTALLMENT_2', 'bigint'),
 ('INSTALLMENT_3', 'bigint'),
 ('INSTALLMENT_AMT_1', 'double'),
 ('INSTALLMENT_AMT_2', 'double'),
 ('INSTALLMENT_AMT_3', 'double'),
 ('INSTALLMENT_PAID_1', 'double'),
 ('INSTALLMENT_PAID_2', 'double'),
 ('INSTALLMENT_PAID_3', 'double'),
 ('default', 'bigint')]

In [211]:
for i in req_columns:
  risk_rank_categorical_columns(traindf,i)

+------+-------------+-----------------+
|   SEX|default_cases|DEFAULT_RISK_RANK|
+------+-------------+-----------------+
|  MALE|         2041|                1|
|FEMALE|         2651|                2|
+------+-------------+-----------------+

+---------------+-------------+-----------------+
|      EDUCATION|default_cases|DEFAULT_RISK_RANK|
+---------------+-------------+-----------------+
|        Unknown|           28|                1|
|    high school|          875|                2|
|graduate school|         1448|                3|
|     university|         2341|                4|
+---------------+-------------+-----------------+

+--------------+-------------+-----------------+
|MARITAL_STATUS|default_cases|DEFAULT_RISK_RANK|
+--------------+-------------+-----------------+
|             0|            5|                1|
|             3|           62|                2|
|       MARRIED|         2294|                3|
|        SINGLE|         2331|                4|
+--------

In [212]:
def ordinal_encoding_test(data,testdata,req_col):
    from pyspark.sql.functions import col
    tmp_data = data.groupby(req_col).agg({'default':'sum'}).withColumnRenamed("sum(default)","occur")
    tmp_data = tmp_data.orderBy('occur')
    from pyspark.sql import Window
    from pyspark.sql.functions import row_number,dense_rank
    # Define a window specification
    windowSpec = Window.orderBy("occur")  # replace "some_column" with a column name
    # Add row numbers
    tmp_data2 = tmp_data.withColumn("rnk", dense_rank().over(windowSpec))
    tmp_data2 = tmp_data2.withColumn("rnk", col("rnk").cast("string"))
    test_keys2 = tmp_data2.select(req_col,'rnk').toPandas()[req_col].tolist()
    test_values2 = tmp_data2.select(req_col,'rnk').toPandas()['rnk'].tolist()
    replacements2 = dict(zip(test_keys2, test_values2))
    data = data.replace(to_replace=replacements2, subset=[req_col])
    data = data.withColumn(req_col, col(req_col).cast("integer"))
    testdata = testdata.replace(to_replace=replacements2, subset=[req_col])
    testdata = testdata.withColumn(req_col, col(req_col).cast("integer"))
    return data,testdata
for i in req_columns:
    traindf,testdf = ordinal_encoding_test(traindf,testdf,i)

In [213]:
traindf_pd = traindf.toPandas()
test_pd = testdf.toPandas()

In [214]:
traindf_pd

,ID,LIMIT_BAL,SEX,EDUCATION,MARITAL_STATUS,AGE,INSTALLMENT_1,INSTALLMENT_2,INSTALLMENT_3,INSTALLMENT_AMT_1,INSTALLMENT_AMT_2,INSTALLMENT_AMT_3,INSTALLMENT_PAID_1,INSTALLMENT_PAID_2,INSTALLMENT_PAID_3,default
0,12194,220000.0,2,3,3,54,1,-2,-1,0.0,0.0,788.0,0.0,788.0,0.0,0
1,7068,90000.0,2,3,4,30,2,2,3,750.0,750.0,750.0,0.0,0.0,0.0,0
2,1446,80000.0,2,4,4,25,2,3,2,69602.0,67930.0,72205.0,0.0,6000.0,2700.0,0
3,318,260000.0,2,2,3,53,1,-1,-1,0.0,165.0,526.0,165.0,526.0,452.0,0
4,13732,30000.0,2,4,4,23,0,0,0,25287.0,26501.0,30300.0,2011.0,5000.0,868.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,16350,210000.0,2,3,4,37,-1,-1,-1,349.0,10128.0,19631.0,10152.0,19631.0,23906.0,1
20996,5187,230000.0,1,3,4,39,0,0,0,38395.0,39943.0,20650.0,2500.0,2000.0,0.0,0
20997,4825,200000.0,1,3,3,31,0,0,0,160292.0,119080.0,113723.0,6036.0,4002.0,4005.0,0
20998,29044,50000.0,2,4,3,50,0,0,0,32570.0,34713.0,39803.0,3000.0,6000.0,3000.0,0


In [215]:
traindf = traindf_pd
testdf = test_pd

In [216]:
testdf

,ID,LIMIT_BAL,SEX,EDUCATION,MARITAL_STATUS,AGE,INSTALLMENT_1,INSTALLMENT_2,INSTALLMENT_3,INSTALLMENT_AMT_1,INSTALLMENT_AMT_2,INSTALLMENT_AMT_3,INSTALLMENT_PAID_1,INSTALLMENT_PAID_2,INSTALLMENT_PAID_3
0,16691,150000.0,1,4,3,36,2,2,0,88655.0,86333.0,88931.0,0.0,4000.0,5000.0
1,15039,50000.0,1,4,4,33,0,0,0,8406.0,8418.0,8347.0,3000.0,3000.0,7508.0
2,7736,30000.0,2,3,4,28,0,0,0,36541.0,17625.0,26243.0,5000.0,11000.0,3000.0
3,20871,200000.0,1,3,4,31,1,2,2,182307.0,186128.0,197509.0,8300.0,15900.0,200.0
4,1651,500000.0,2,3,3,45,-1,-1,-1,36954.0,-24704.0,22999.0,0.0,83000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,11671,20000.0,1,4,3,31,0,0,0,18490.0,25404.0,19789.0,1500.0,2393.0,1000.0
8996,29527,60000.0,1,2,4,46,0,0,0,61548.0,63020.0,16982.0,2700.0,1500.0,2000.0
8997,9016,70000.0,1,2,3,56,0,0,0,42039.0,46330.0,47480.0,5000.0,2238.0,3000.0
8998,28520,80000.0,2,4,4,24,0,0,0,19188.0,18774.0,12339.0,1374.0,1235.0,1101.0


In [217]:
y = traindf['default']
X = traindf[traindf.drop('default',axis = 1).columns]


In [218]:
!pip install 'aif360[inFairness]' --q

In [219]:
X

,ID,LIMIT_BAL,SEX,EDUCATION,MARITAL_STATUS,AGE,INSTALLMENT_1,INSTALLMENT_2,INSTALLMENT_3,INSTALLMENT_AMT_1,INSTALLMENT_AMT_2,INSTALLMENT_AMT_3,INSTALLMENT_PAID_1,INSTALLMENT_PAID_2,INSTALLMENT_PAID_3
0,12194,220000.0,2,3,3,54,1,-2,-1,0.0,0.0,788.0,0.0,788.0,0.0
1,7068,90000.0,2,3,4,30,2,2,3,750.0,750.0,750.0,0.0,0.0,0.0
2,1446,80000.0,2,4,4,25,2,3,2,69602.0,67930.0,72205.0,0.0,6000.0,2700.0
3,318,260000.0,2,2,3,53,1,-1,-1,0.0,165.0,526.0,165.0,526.0,452.0
4,13732,30000.0,2,4,4,23,0,0,0,25287.0,26501.0,30300.0,2011.0,5000.0,868.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,16350,210000.0,2,3,4,37,-1,-1,-1,349.0,10128.0,19631.0,10152.0,19631.0,23906.0
20996,5187,230000.0,1,3,4,39,0,0,0,38395.0,39943.0,20650.0,2500.0,2000.0,0.0
20997,4825,200000.0,1,3,3,31,0,0,0,160292.0,119080.0,113723.0,6036.0,4002.0,4005.0
20998,29044,50000.0,2,4,3,50,0,0,0,32570.0,34713.0,39803.0,3000.0,6000.0,3000.0


In [220]:
y

0        0
1        0
2        0
3        0
4        0
        ..
20995    1
20996    0
20997    0
20998    0
20999    0
Name: default, Length: 21000, dtype: int64

In [221]:
import numpy as np
import pandas as pd

In [222]:
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import StandardScaler

# Convert your data to a BinaryLabelDataset
dataset = BinaryLabelDataset(df=pd.concat([X, y], axis=1), label_names=['default'], protected_attribute_names=['SEX'])

# Identify the privileged and unprivileged groups
Privileged_groups = [{'SEX': 2}] # Change this based on your data(female)
Unprivileged_groups = [{'SEX': 1}] # Change this based on your data (male)

# Create the Reweighing object
RW = Reweighing(unprivileged_groups=Unprivileged_groups, privileged_groups=Privileged_groups)

# Fit and transform the original dataset
dataset_transf = RW.fit_transform(dataset)

# You can convert it back to a DataFrame
fair_df = pd.DataFrame(data=dataset_transf.features, columns=X.columns)
fair_labels = pd.DataFrame(data=dataset_transf.labels, columns=['default'])


In [223]:
fair_labels.value_counts()

default
0.0        16308
1.0         4692
dtype: int64

In [224]:
y.value_counts()

0    16308
1     4692
Name: default, dtype: int64

In [225]:
fair_df_spark = spark.createDataFrame(fair_df)
fair_labels_spark = spark.createDataFrame(fair_labels)

In [226]:
fair_df_spark.show()

+-------+---------+---+---------+--------------+----+-------------+-------------+-------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|     ID|LIMIT_BAL|SEX|EDUCATION|MARITAL_STATUS| AGE|INSTALLMENT_1|INSTALLMENT_2|INSTALLMENT_3|INSTALLMENT_AMT_1|INSTALLMENT_AMT_2|INSTALLMENT_AMT_3|INSTALLMENT_PAID_1|INSTALLMENT_PAID_2|INSTALLMENT_PAID_3|
+-------+---------+---+---------+--------------+----+-------------+-------------+-------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|12194.0| 220000.0|2.0|      3.0|           3.0|54.0|          1.0|         -2.0|         -1.0|              0.0|              0.0|            788.0|               0.0|             788.0|               0.0|
| 7068.0|  90000.0|2.0|      3.0|           4.0|30.0|          2.0|          2.0|          3.0|            750.0|            750.0|            750.0|               0.0|    

In [227]:
from pyspark.sql.functions import monotonically_increasing_id
fair_df_spark = fair_df_spark.withColumn("_c0", monotonically_increasing_id())
fair_labels_spark = fair_labels_spark.withColumn("_c0", monotonically_increasing_id())

In [228]:
fair_df_spark = fair_df_spark.join(fair_labels_spark, on='_c0')

In [229]:
from pyspark.sql.functions import col
fair_df_spark = fair_df_spark.withColumn("SEX", col("SEX").cast("string"))

fair_df_spark = fair_df_spark.withColumn("default", col("default").cast("string"))

In [230]:
from pyspark.sql.functions import concat,lit
fair_df_spark2= fair_df_spark.withColumn('strata', concat(col('SEX'), lit('-'), col('default')))

In [231]:
fair_df_spark2.show()

+----+-------+---------+---+---------+--------------+----+-------------+-------------+-------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-------+-------+
| _c0|     ID|LIMIT_BAL|SEX|EDUCATION|MARITAL_STATUS| AGE|INSTALLMENT_1|INSTALLMENT_2|INSTALLMENT_3|INSTALLMENT_AMT_1|INSTALLMENT_AMT_2|INSTALLMENT_AMT_3|INSTALLMENT_PAID_1|INSTALLMENT_PAID_2|INSTALLMENT_PAID_3|default| strata|
+----+-------+---------+---+---------+--------------+----+-------------+-------------+-------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-------+-------+
|  26|28996.0| 100000.0|2.0|      4.0|           4.0|40.0|          2.0|          0.0|          0.0|          40626.0|          41945.0|          45247.0|            2000.0|            4000.0|            1000.0|    1.0|2.0-1.0|
|  29|29660.0|  50000.0|1.0|      4.0|           3.0|37.0|          0.0|          0.0|  

In [232]:
fair_df_spark2 = fair_df_spark2.withColumn("SEX", col("SEX").cast("int"))

fair_df_spark2 = fair_df_spark2.withColumn("default", col("default").cast("int"))

In [233]:
FAIR = fair_df_spark2

In [238]:
FAIR= pd.read_csv('https://raw.githubusercontent.com/Harshit-Budakoti/enigma_hb/main/fairdata_comb.csv')


In [240]:
FAIR=FAIR.drop('_c0',axis = 1)

In [241]:
fair_df_new = FAIR.drop(['strata'],axis = 1 )

fair_labels_new = fair_labels

In [250]:
fair_df_new

,ID,LIMIT_BAL,SEX,EDUCATION,MARITAL_STATUS,AGE,INSTALLMENT_1,INSTALLMENT_2,INSTALLMENT_3,INSTALLMENT_AMT_1,INSTALLMENT_AMT_2,INSTALLMENT_AMT_3,INSTALLMENT_PAID_1,INSTALLMENT_PAID_2,INSTALLMENT_PAID_3,default
0,12194.0,220000.0,2,3.0,3.0,54.0,1.0,-2.0,-1.0,0.0,0.0,788.0,0.0,788.0,0.0,0
1,7068.0,90000.0,2,3.0,4.0,30.0,2.0,2.0,3.0,750.0,750.0,750.0,0.0,0.0,0.0,0
2,1446.0,80000.0,2,4.0,4.0,25.0,2.0,3.0,2.0,69602.0,67930.0,72205.0,0.0,6000.0,2700.0,0
3,318.0,260000.0,2,2.0,3.0,53.0,1.0,-1.0,-1.0,0.0,165.0,526.0,165.0,526.0,452.0,0
4,13732.0,30000.0,2,4.0,4.0,23.0,0.0,0.0,0.0,25287.0,26501.0,30300.0,2011.0,5000.0,868.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,16350.0,210000.0,2,3.0,4.0,37.0,-1.0,-1.0,-1.0,349.0,10128.0,19631.0,10152.0,19631.0,23906.0,1
20996,5187.0,230000.0,1,3.0,4.0,39.0,0.0,0.0,0.0,38395.0,39943.0,20650.0,2500.0,2000.0,0.0,0
20997,4825.0,200000.0,1,3.0,3.0,31.0,0.0,0.0,0.0,160292.0,119080.0,113723.0,6036.0,4002.0,4005.0,0
20998,29044.0,50000.0,2,4.0,3.0,50.0,0.0,0.0,0.0,32570.0,34713.0,39803.0,3000.0,6000.0,3000.0,0


In [242]:
fair_df_newest = fair_df_new.drop('default',axis = 1)

In [251]:
fair_df_newest

,ID,LIMIT_BAL,SEX,EDUCATION,MARITAL_STATUS,AGE,INSTALLMENT_1,INSTALLMENT_2,INSTALLMENT_3,INSTALLMENT_AMT_1,INSTALLMENT_AMT_2,INSTALLMENT_AMT_3,INSTALLMENT_PAID_1,INSTALLMENT_PAID_2,INSTALLMENT_PAID_3
0,12194.0,220000.0,2,3.0,3.0,54.0,1.0,-2.0,-1.0,0.0,0.0,788.0,0.0,788.0,0.0
1,7068.0,90000.0,2,3.0,4.0,30.0,2.0,2.0,3.0,750.0,750.0,750.0,0.0,0.0,0.0
2,1446.0,80000.0,2,4.0,4.0,25.0,2.0,3.0,2.0,69602.0,67930.0,72205.0,0.0,6000.0,2700.0
3,318.0,260000.0,2,2.0,3.0,53.0,1.0,-1.0,-1.0,0.0,165.0,526.0,165.0,526.0,452.0
4,13732.0,30000.0,2,4.0,4.0,23.0,0.0,0.0,0.0,25287.0,26501.0,30300.0,2011.0,5000.0,868.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,16350.0,210000.0,2,3.0,4.0,37.0,-1.0,-1.0,-1.0,349.0,10128.0,19631.0,10152.0,19631.0,23906.0
20996,5187.0,230000.0,1,3.0,4.0,39.0,0.0,0.0,0.0,38395.0,39943.0,20650.0,2500.0,2000.0,0.0
20997,4825.0,200000.0,1,3.0,3.0,31.0,0.0,0.0,0.0,160292.0,119080.0,113723.0,6036.0,4002.0,4005.0
20998,29044.0,50000.0,2,4.0,3.0,50.0,0.0,0.0,0.0,32570.0,34713.0,39803.0,3000.0,6000.0,3000.0


In [252]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import classification_report
from imblearn.ensemble import BalancedRandomForestClassifier



# Define transformers for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())
])

# Create a ColumnTransformer to handle numeric features
preprocessor = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer, fair_df_newest.columns)
])



# Create the BalancedRandomForestClassifier
brfc2000 = BalancedRandomForestClassifier(n_estimators=2000,max_depth = 8)
brfc1000 = BalancedRandomForestClassifier(n_estimators=1000,max_depth = 8)
brfc1500 = BalancedRandomForestClassifier(n_estimators=1500,max_depth = 8)
brfc1250 = BalancedRandomForestClassifier(n_estimators=1250,max_depth = 8)
brfc1750 = BalancedRandomForestClassifier(n_estimators=1750,max_depth = 8)

# Combine SMOTE with the preprocessor and the classifier in a pipeline
pipeline2000 = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', brfc2000)
])
pipeline1000 = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', brfc1000)
])
pipeline1500 = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', brfc1500)
])
pipeline1250 = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', brfc1250)
])
pipeline1750 = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', brfc1750)
])

In [253]:
import random

In [254]:
X_train, X_val, y_train, y_val = train_test_split(fair_df_newest, fair_labels_new, test_size=0.43,stratify = FAIR.strata,random_state = random.randint(1,10000))

In [255]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import numpy as np

# Define the ensemble model
ensemble = VotingClassifier(estimators=[('pipeline1', pipeline1000), ('pipeline2', pipeline2000),('pipeline3', pipeline1500),
                                        ('pipeline4', pipeline1250),('pipeline5', pipeline1750)], voting='soft')

# Fit the ensemble model
ensemble.fit(X_train, y_train)

# Predict the probabilities of the positive class
y_val_probs = ensemble.predict_proba(X_val)[:, 1]

# Define a range of thresholds to test
thresholds = np.linspace(0, 1, 500)

best_threshold = None
best_score = -np.inf

# Loop over the thresholds
for threshold in thresholds:
    # Apply the threshold
    y_val_pred = (y_val_probs > threshold).astype(int)

    # Calculate the f1 score
    f1 = f1_score(y_val, y_val_pred)

    # If this score is better than the previous best, update the best score and the best threshold
    if f1 > best_score:
        best_score = f1
        best_threshold = threshold

print(f"The optimal threshold is {best_threshold} with an F1 score of {best_score}")


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/util

The optimal threshold is 0.561122244488978 with an F1 score of 0.5395487230349616


In [246]:
!pip install fairlearn --q

In [256]:
import statistics
from fairlearn.metrics import demographic_parity_ratio
import scipy
from scipy.stats import hmean

In [257]:
# Initialize the lists to store the optimal thresholds and best scores
optimal_thresholds_1 = []
optimal_thresholds_2 = []
best_scores_1 = []
best_scores_2 = []
harmonic_means = []
f1_scores= []

# Perform the operation 5 times
for i in range(5):
    # Split data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(fair_df_newest, fair_labels_new, test_size=0.43, random_state=random.randint(1,10000),stratify = FAIR.strata)
    model = ensemble.fit(X_train,y_train)
    y_val_pred = model.predict_proba(X_val)[:, 1]  # Predicted probabilities for positive class

    for gender in [1, 2]:
        # Filter the data based on the gender
        X_val_gender = X_val[X_val['SEX'] == gender]
        y_val_gender = y_val[X_val['SEX'] == gender]
        y_val_pred_gender = y_val_pred[X_val['SEX'] == gender]

        # Define the sensitive features
        Sensitive_features = X_val_gender['SEX'].replace({1:0, 2:1})

        # Define a range of thresholds to test
        thresholds = np.linspace(0, 1, 100)

        best_threshold = None
        best_score = -np.inf

        # Loop over the thresholds
        for threshold in thresholds:
            # Apply the threshold
            y_pred_thresholded = (y_val_pred_gender > threshold).astype(int)

            # Calculate the f1 score
            f1 = f1_score(y_val_gender, y_pred_thresholded)

            # Calculate the demographic parity score
            dps = demographic_parity_ratio(y_val_gender, y_pred_thresholded, sensitive_features=Sensitive_features)

            # Calculate the harmonic mean of the f1 score and the demographic parity score
            score = hmean([f1, dps])

            # If this score is better than the previous best, update the best score and the best threshold
            if score > best_score:
                best_score = score
                best_threshold = threshold

        # Store the best threshold and best score
        if gender == 1:
            optimal_thresholds_1.append(best_threshold)
            best_scores_1.append(best_score)
        else:
            optimal_thresholds_2.append(best_threshold)
            best_scores_2.append(best_score)

        # Apply the best threshold to the predicted probabilities
        iteration_labels = np.where(y_val_pred_gender >= best_threshold, 1, 0)

        # Calculate the f1 score and demographic parity ratio
        f1 = f1_score(y_val_gender, iteration_labels)
        dps = demographic_parity_ratio(y_val_gender, iteration_labels, sensitive_features=Sensitive_features)
        f1_scores.append(f1)
        # Calculate the harmonic mean and store it
        harmonic_means.append(hmean([f1, dps]))



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/util

In [249]:
from scipy import stats

In [258]:
# Calculate the average of the optimal thresholds, best scores, and harmonic means
average_threshold_1 = stats.hmean(optimal_thresholds_1)
average_threshold_2 = stats.hmean(optimal_thresholds_2)
average_best_score_1 = stats.hmean(best_scores_1)
average_best_score_2 = stats.hmean(best_scores_2)
average_harmonic_mean = stats.hmean(harmonic_means)
average_f1_score = stats.hmean(f1_scores)

print(f"The average optimal threshold for gender 1 is {average_threshold_1}")
print(f"The average optimal threshold for gender 2 is {average_threshold_2}")
print(f"The average best score for gender 1 is {average_best_score_1}")
print(f"The average best score for gender 2 is {average_best_score_2}")
print(f"The average harmonic mean is {average_harmonic_mean}")
print(f"The average f1 score is {average_f1_score}")


The average optimal threshold for gender 1 is 0.5145820393822423
The average optimal threshold for gender 2 is 0.5258084304618351
The average best score for gender 1 is 0.715198882267856
The average best score for gender 2 is 0.6987852802421886
The average harmonic mean is 0.7068968162801379
The average f1 score is 0.5466669830992235


In [259]:
optimal_thresholds_1

[0.5252525252525253,
 0.5151515151515152,
 0.5151515151515152,
 0.48484848484848486,
 0.5353535353535354]

In [261]:
optimal_thresholds_2

[0.48484848484848486,
 0.5050505050505051,
 0.5555555555555556,
 0.5454545454545455,
 0.5454545454545455]

# **NOTE:  Here Gender 1 is Male and Gender 2 is Female. The final csv is named submission_final and will be downloaded locally on the execution of entire notebook**

In [ ]:
#

In [262]:
votingmodel = ensemble.fit(fair_df_new.drop('default',axis=1),fair_labels_new)
y_pred_voting = votingmodel.predict_proba(testdf)[:, 1]  # Predicted probabilities for positive class

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/util

In [ ]:
pd.DataFrame(y_pred_voting).to_csv('y_pred_voting_prob.csv',index = True)
from google.colab import files
files.download('y_pred_voting_prob.csv')

In [263]:
voting_labels = np.where((testdf['SEX'] == 1) & (y_pred_voting >= average_threshold_1), 1,
                            np.where((testdf['SEX'] == 2) & (y_pred_voting >= average_threshold_2), 1, 0))

In [264]:
voting_labels

array([1, 0, 0, ..., 0, 0, 0])

In [265]:
pd.DataFrame(voting_labels).value_counts()

0    6775
1    2225
dtype: int64

In [266]:
submission_final= pd.DataFrame({'ID':testdf.ID,'DEFAULT.PAYMENT.NEXT.MONTH':voting_labels})


In [269]:
submission_final

,ID,DEFAULT.PAYMENT.NEXT.MONTH
0,16691,1
1,15039,0
2,7736,0
3,20871,1
4,1651,0
...,...,...
8995,11671,0
8996,29527,0
8997,9016,0
8998,28520,0


In [ ]:
spark.write.format()

In [272]:
import pandas as pd

In [274]:
submission_final_sp = spark.createDataFrame(submission_final)

In [276]:
submission_final_sp.show()

+-----+--------------------------+
|   ID|DEFAULT.PAYMENT.NEXT.MONTH|
+-----+--------------------------+
|16691|                         1|
|15039|                         0|
| 7736|                         0|
|20871|                         1|
| 1651|                         0|
|21031|                         0|
| 2890|                         0|
| 8417|                         0|
|21891|                         0|
|29776|                         1|
| 2414|                         0|
|16525|                         1|
|26280|                         1|
|  647|                         1|
| 2717|                         0|
|22536|                         0|
|13935|                         0|
|22863|                         1|
| 3722|                         1|
|11983|                         0|
+-----+--------------------------+
only showing top 20 rows



In [277]:
submission_final_sp.write.csv('/content/final_submit.csv')

In [281]:
submission_final_sp.coalesce(1).write.format('csv').option('header','true').save('/content/drive/My Drive/enigma/final_submission.csv')

In [273]:
# Assuming you have a DataFrame called 'my_dataframe'
submission_final.to_csv('submission_final.csv', index=False)


ImportError: cannot import name 'SequenceNotStr' from 'pandas._typing' (/usr/local/lib/python3.10/dist-packages/pandas/_typing.py)